In [ ]:
import os
import sys
import copy
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

In [ ]:
%load_ext autoreload
%aimport smooth.config
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
dataset = smooth.datasets.from_params("mnist12")

In [ ]:
model_1 = smooth.model.train_shallow(
    dataset,
    learning_rate=0.01,
    init_scale=0.1,
    epochs=200,
    verbose=2,
    batch_size=64,
    gradient_norm_squared_reg_coef=1e-1,
)

In [ ]:
smooth.measures.get_measures(model_1, dataset)

In [ ]:
x1 = tf.constant([dataset.x_train[0]])
x2 = tf.constant([dataset.x_train[1]])

x1 = tf.constant([[0., 2.], [1., 1.]])
x2 = tf.constant([[2., 0.], [2., 2.]])

In [ ]:
n_segments = len(x1)
n_samples = 4
samples = smooth.measures._interpolate(x1, x2, n_samples)

samples_flat = tf.reshape(samples, (n_samples * n_segments,) + samples.shape[2:])

In [ ]:
tf.tensordot(w, x1, axes=0) + tf.tensordot(1-w, x2, axes=0)

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(x1)
    y = smooth.measures._path_length(x1, batch=False)
    y = tf.expand_dims(y, -1)

tape.jacobian(y, x1)
# print(y)